# Deep Learning with Python

## 2.3 The gears of neural networks: tensor operations

> 神经网络的“齿轮”: 张量运算

在我们的第一个神经网络例子中(MNIST)，我们的每一层其实都是对输入数据做了类似如下的运算：

```
output = relu(dot(W, input) + b)
```

input 是输入，
W 和 b 是层的属性，
output 是输出。

这些东西之间做了 relu、dot、add 运算，
接下来我们会解释这些运算。

### 逐元素操作(Element-wise)

Element-wise 的操作，就是分别对张量中的每一个元素作用。
比如，我们实现一个简单的 `relu` （`relu(x) = max(x, 0)`）:

In [2]:
def naive_relu(x):
    assert len(x.shape) == 2    # x is a 2D Numpy tensor.
    x = x.copy()    # Avoid overwriting the input tensor.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

加法也是逐元素操作：

In [3]:
def naive_add(x, y):
    # assert x and y are 2D Numpy tensors and have the same shape.
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()    # Avoid overwriting the input tensor.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

在 Numpy 里，这些都写好了。 具体的运算是交给 C 或 Fortran 写的 BLAS 进行的，速度很高。

你可以这样查看有没有装 BLAS：

In [5]:
import numpy as np

np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]


下面是如何使用 numpy 的逐元素 relu、add：

In [7]:
a = np.array([[1, 2, 3],
              [-1, 2, -3],
              [3, -1, 4]])
b = np.array([[6, 7, 8], 
              [-2, -3, 1], 
              [1, 0, 4]])

c = a + b    # Element-wise addition
d = np.maximum(c, 0)    # Element-wise relu

print(c)
print(d)

[[ 7  9 11]
 [-3 -1 -2]
 [ 4 -1  8]]
[[ 7  9 11]
 [ 0  0  0]
 [ 4  0  8]]


### 广播(Broadcasting)

当进行逐元素运算时，如果两个张量的形状不同，在可行的情况下，较小的张量会「广播」成和较大的张量一样的形状。

具体来说，可以通过广播，对形状为 `(a, b, ..., n, n+1, ..., m)` 和 `(n, n+1, ..., m)` 的两个张量进行逐元素运算。

比如：

In [9]:
x = np.random.random((64, 3, 32, 10))    # x is a random tensor with shape (64, 3, 32, 10).
y = np.random.random((32, 10))    # y is a random tensor with shape (32, 10).
z = np.maximum(x, y)    # The output z has shape (64, 3, 32, 10) like x.

广播的操作如下：

1. 小张量增加轴（广播轴），加到和大的一样（ndim）
2. 小张量的元素在新轴上重复，加到和大的一样（shape）

E.g. 
        
    x: (32, 10), y: (10,)
    Step 1: add an empty first axis to y: Y -> (1, 10)
    Step 2: repeat y 32 times alongside this new axis: Y -> (32, 10)

在完成后，有 `Y[i, :] == y for i in range(0, 32)`

当然，在实际的实现里，我们不这样去复制，这样太浪费空间了，
我们是直接在算法里实现这个“复制的”。
比如，我们实现一个简单的向量和矩阵相加：

In [10]:
def naive_add_matrix_and_vector(m, v):
    assert len(m.shape) == 2    # m is a 2D Numpy tensor.
    assert len(v.shape) == 1    # v is a Numpy vector.
    assert m.shape[1] == v.shape[0]
    
    m = m.copy()
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            m[i, j] += v[j]
    return m

naive_add_matrix_and_vector(np.array([[1 ,2, 3], [4, 5, 6], [7, 8, 9]]), 
                            np.array([1, -1, 100]))

array([[  2,   1, 103],
       [  5,   4, 106],
       [  8,   7, 109]])

### 张量点积(dot)

张量点积，或者叫张量乘积，在 numpy 里用 `dot(x, y)` 完成。

点积的操作可以从如下的简单程序中看出：

In [12]:
# 向量点积
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z


# 矩阵与向量点积
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z


# 矩阵点积
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

In [13]:
a = np.array([[1, 2, 3],
              [-1, 2, -3],
              [3, -1, 4]])
b = np.array([[6, 7, 8], 
              [-2, -3, 1], 
              [1, 0, 4]])
naive_matrix_dot(a, b)

array([[  5.,   1.,  22.],
       [-13., -13., -18.],
       [ 24.,  24.,  39.]])

对于高维的张量点积，其实也是一样的。
例如，(这说的是shape哈)：

```
(a, b, c, d) . (d,) -> (a, b, c)
(a, b, c, d) . (d, e) -> (a, b, c, e)
```

### 张量变形(reshaping)

这个操作，简言之就是，，，还是那些元素，只是排列的方式变了。

In [15]:
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])
print(x.shape)

(3, 2)


In [16]:
x.reshape((6, 1))

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [17]:
x.reshape((2, 3))

array([[0., 1., 2.],
       [3., 4., 5.]])

「转置」(transposition) 是一种特殊的矩阵变形，
转置就是行列互换。

原来的 `x[i, :]`，转置后就成了 `x[:, i]`。

In [19]:
x = np.zeros((300, 20))
y = np.transpose(x)
print(y.shape)

(20, 300)
